In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def accuracy(actual, predictions):
    logr_accuracy_all_predictors = np.mean(actual == predictions)
    return logr_accuracy_all_predictors  # Corrected indentation

def calibration(actual, predictions):
    return np.sum(predictions) / np.sum(actual)

def logloss(actual, predictions):
    epsilon = 1e-15
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    
    logr_logloss_all_predictors = -(1 / len(actual)) * np.sum(
        actual * np.log(predictions) + (1 - actual) * np.log(1 - predictions))
    return logr_logloss_all_predictors

In [3]:
def calculate_implied_probabilities(odds_w, odds_l):
    if odds_w == 0 or odds_l == 0: #or pd.isna(odds_w) or pd.isna(odds_l):
        odds_w = odds_w + 0.001
        odds_l = odds_l + 0.001


    implied_prob_w = 1 / odds_w
    implied_prob_l = 1 / odds_l
    overround = implied_prob_w + implied_prob_l
    normalized_prob_w = implied_prob_w / overround
    normalized_prob_l = implied_prob_l / overround
    return normalized_prob_w, normalized_prob_l

In [4]:
tennis_dfs = {}

In [5]:
for year in range(2000, 2020): 
    file_extension = 'xls' if year < 2013 else 'xlsx'
    file_path = f"Betting_Odds_Tennis/{year}.{file_extension}"
    tennis_dfs[year] = pd.read_excel(file_path)

/Users/harishthota/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/harishthota/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/harishthota/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/harishthota/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [6]:
tennis_dfs

{2000:       ATP  Location                          Tournament       Date  \
 0       1  Adelaide  Australian Hardcourt Championships 2000-01-03   
 1       1  Adelaide  Australian Hardcourt Championships 2000-01-03   
 2       1  Adelaide  Australian Hardcourt Championships 2000-01-03   
 3       1  Adelaide  Australian Hardcourt Championships 2000-01-03   
 4       1  Adelaide  Australian Hardcourt Championships 2000-01-03   
 ...   ...       ...                                 ...        ...   
 2958   69    Lisbon                         Masters Cup 2000-11-27   
 2959   69    Lisbon                         Masters Cup 2000-11-27   
 2960   69    Lisbon                         Masters Cup 2000-11-27   
 2961   69    Lisbon                         Masters Cup 2000-11-27   
 2962   69    Lisbon                         Masters Cup 2000-11-27   
 
              Series    Court Surface        Round  Best of       Winner  ...  \
 0     International  Outdoor    Hard    1st Round        3

In [7]:
all_years_df = pd.concat(tennis_dfs.values(), ignore_index=True)

In [8]:
all_years_df

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54903,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Nadal R.,...,NaN,NaN,NaN,NaN,NaN,NaN,1.48,3.30,1.41,2.93
54904,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,...,NaN,NaN,NaN,NaN,NaN,NaN,2.24,2.06,1.92,1.90
54905,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Tsitsipas S.,...,NaN,NaN,NaN,NaN,NaN,NaN,3.75,1.40,3.39,1.33
54906,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Thiem D.,...,NaN,NaN,NaN,NaN,NaN,NaN,1.87,2.20,1.78,2.06


In [9]:
all_years_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
       'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
       'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL'],
      dtype='object')

In [10]:
odds_columns = ['B365W', 'B365L', 'B&WW', 'B&WL', 'CBW', 'CBL', 'EXW', 'EXL', 'LBW', 'LBL',
                'GBW', 'GBL', 'IWW', 'IWL', 'PSW', 'PSL', 'SBW', 'SBL', 'SJW', 'SJL',
                'UBW', 'UBL', 'MaxW', 'MaxL', 'AvgW', 'AvgL']
essential_columns = ['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'Wsets', 'Lsets']

all_years_df['EXW'] = all_years_df['EXW'].replace('2.,3', 2.3)

all_years_df['EXW'].unique()

array([   nan,  1.48 ,  1.2  ,  1.5  ,  1.8  ,  1.9  ,  1.75 ,  1.65 ,
        1.55 ,  2.45 ,  3.12 ,  2.67 ,  2.1  ,  2.15 ,  1.28 ,  2.   ,
        2.6  ,  1.7  ,  1.85 ,  2.3  ,  2.71 ,  1.45 ,  1.15 ,  1.17 ,
        1.71 ,  1.42 ,  2.53 ,  1.6  ,  1.3  ,  1.23 ,  1.25 ,  1.4  ,
        2.85 ,  1.1  ,  1.35 ,  1.95 ,  3.1  ,  4.25 ,  2.31 ,  2.35 ,
        2.92 ,  1.11 ,  3.46 ,  1.05 ,  2.05 ,  2.2  ,  7.25 ,  1.37 ,
        1.67 ,  3.98 ,  1.14 ,  3.35 ,  3.81 ,  2.75 ,  1.18 ,  3.75 ,
        1.78 ,  1.02 ,  1.22 ,  2.56 ,  2.34 ,  1.44 ,  1.07 ,  1.32 ,
        1.21 ,  2.25 ,  1.04 ,  1.06 ,  1.13 ,  2.17 ,  1.26 ,  1.53 ,
        2.83 ,  1.19 ,  4.67 ,  1.27 ,  1.47 ,  2.28 ,  2.65 ,  1.03 ,
        1.39 ,  2.4  ,  1.49 ,  3.   ,  1.76 ,  1.58 ,  1.36 ,  1.63 ,
        1.34 ,  5.2  ,  1.33 ,  3.22 ,  2.03 ,  3.05 ,  6.75 ,  4.06 ,
        2.9  ,  5.15 ,  1.12 ,  2.5  ,  2.8  ,  1.81 ,  1.08 ,  3.01 ,
        1.38 ,  1.24 ,  6.25 ,  5.85 ,  3.45 ,  2.12 ,  1.91 ,  1.56 ,
      

In [11]:
# for col in odds_columns:
#     print(col)
#     all_years_df[col].fillna(all_years_df[col].mean(), inplace=True)

# # Define a high arbitrary value to replace NaN values
# high_value = 0.0001

# # Fill NaN values with the high arbitrary value
# for col in odds_columns:
#     all_years_df[col].fillna(high_value, inplace=True)

In [12]:
# Strategy 1: Always bet on the player with the smallest odd
def smallest_odd_strategy(row, bookmaker):
    winner = row['Winner']
    loser = row['Loser']
    winner_odd = row[f'{bookmaker}W']
    loser_odd = row[f'{bookmaker}L']
    if winner_odd < loser_odd:
        return winner, winner_odd
    else:
        return loser, loser_odd


# Strategy 2: Always bet on the player with the best ATP ranking
def best_ranking_strategy(row, bookmaker):
    winner = row['Winner']
    loser = row['Loser']
    winner_rank = row['WRank']
    loser_rank = row['LRank']
    if winner_rank < loser_rank:
        return winner, row['WRank']
    else:
        return loser, row['LRank']
    

# Strategy 3: Bet completely at random
def random_strategy(row, bookmaker):
    if np.random.rand() > 0.5:
        return row['Winner'], row['WRank']
    else:
        return row['Loser'], row['LRank']

In [13]:
# # Evaluate strategies
# def evaluate_strategy(strategy_func, bookmaker=None):
#     results = []
#     for idx, row in df_odds.iterrows():
#         if bookmaker:
#             player, odd = strategy_func(row, bookmaker)
#         else:
#             player, odd = strategy_func(row)
#         if player == row['Winner']:
#             results.append(odd - 1)  # Win: payout is the odd minus the stake (1 unit)
#         else:
#             results.append(-1)  # Lose: loss is the stake (1 unit)
#     return np.cumsum(results)

# # Apply strategies for two bookmakers (e.g., Bet365 and Expekt)
# bookmakers = ['B365', 'EX']
# strategies = {
#     'Smallest Odd': smallest_odd_strategy,
#     'Best ATP Ranking': best_ranking_strategy,
#     'Random': random_strategy
# }

# # Store cumulative returns
# cumulative_returns = {}
# for bookmaker in bookmakers:
#     for strategy_name, strategy_func in strategies.items():
#         key = f'{strategy_name} ({bookmaker})'
#         if strategy_name == 'Random':
#             # Average over multiple random runs for stability
#             returns = np.mean([evaluate_strategy(strategy_func) for _ in range(100)], axis=0)
#         else:
#             returns = evaluate_strategy(strategy_func, bookmaker)
#         cumulative_returns[key] = returns

# # Convert to DataFrame for plotting
# returns_df = pd.DataFrame(cumulative_returns)


In [14]:
def evaluate_strategy(strategy_func, bookmaker=None):
    results = []
    for idx, row in df_odds.iterrows():
        if bookmaker:
            player, odd = strategy_func(row, bookmaker)
        else:
            player, odd = strategy_func(row)
        if player == row['Winner']:
            results.append(odd - 1)  # Win: payout is the odd minus the stake (1 unit)
        else:
            results.append(-1)  # Lose: loss is the stake (1 unit)
    return np.cumsum(results)

In [15]:
# # Plot cumulative returns
# plt.figure(figsize=(14, 8))
# for col in returns_df.columns:
#     plt.plot(returns_df[col], label=col)
# plt.xlabel('Match Number')
# plt.ylabel('Cumulative Return')
# plt.title('Comparison of Betting Strategies for Two Bookmakers')
# plt.legend()
# plt.show()

In [16]:
df_odds = all_years_df[essential_columns + odds_columns].copy()
df_odds

df_odds.fillna(df_odds.mean(), inplace=True)

/var/folders/4r/pq79tqfn6hd9mhpz_xr5ky1w0000gn/T/ipykernel_5442/1756135060.py:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_odds.fillna(df_odds.mean(), inplace=True)
/var/folders/4r/pq79tqfn6hd9mhpz_xr5ky1w0000gn/T/ipykernel_5442/1756135060.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_odds.fillna(df_odds.mean(), inplace=True)


In [17]:
df_odds['WRank'].replace('NR', np.nan, inplace=True)
df_odds['LRank'].replace('NR', np.nan, inplace=True)

df_odds['WRank'].fillna(100000, inplace=True)
df_odds['LRank'].fillna(100000, inplace=True)

In [18]:
all_years_df['EXW'].unique()

array([   nan,  1.48 ,  1.2  ,  1.5  ,  1.8  ,  1.9  ,  1.75 ,  1.65 ,
        1.55 ,  2.45 ,  3.12 ,  2.67 ,  2.1  ,  2.15 ,  1.28 ,  2.   ,
        2.6  ,  1.7  ,  1.85 ,  2.3  ,  2.71 ,  1.45 ,  1.15 ,  1.17 ,
        1.71 ,  1.42 ,  2.53 ,  1.6  ,  1.3  ,  1.23 ,  1.25 ,  1.4  ,
        2.85 ,  1.1  ,  1.35 ,  1.95 ,  3.1  ,  4.25 ,  2.31 ,  2.35 ,
        2.92 ,  1.11 ,  3.46 ,  1.05 ,  2.05 ,  2.2  ,  7.25 ,  1.37 ,
        1.67 ,  3.98 ,  1.14 ,  3.35 ,  3.81 ,  2.75 ,  1.18 ,  3.75 ,
        1.78 ,  1.02 ,  1.22 ,  2.56 ,  2.34 ,  1.44 ,  1.07 ,  1.32 ,
        1.21 ,  2.25 ,  1.04 ,  1.06 ,  1.13 ,  2.17 ,  1.26 ,  1.53 ,
        2.83 ,  1.19 ,  4.67 ,  1.27 ,  1.47 ,  2.28 ,  2.65 ,  1.03 ,
        1.39 ,  2.4  ,  1.49 ,  3.   ,  1.76 ,  1.58 ,  1.36 ,  1.63 ,
        1.34 ,  5.2  ,  1.33 ,  3.22 ,  2.03 ,  3.05 ,  6.75 ,  4.06 ,
        2.9  ,  5.15 ,  1.12 ,  2.5  ,  2.8  ,  1.81 ,  1.08 ,  3.01 ,
        1.38 ,  1.24 ,  6.25 ,  5.85 ,  3.45 ,  2.12 ,  1.91 ,  1.56 ,
      

In [19]:
for bookmaker in ['B365', 'B&W', 'CB', 'EX', 'LB', 'GB', 'IW', 'PS', 'SB', 'SJ', 'UB', 'Max', 'Avg']:
    print(bookmaker)
    df_odds[f'{bookmaker}_prob_w'], df_odds[f'{bookmaker}_prob_l'] = zip(*df_odds.apply(lambda row: calculate_implied_probabilities(row[f'{bookmaker}W'], row[f'{bookmaker}L']), axis=1))

B365
B&W
CB
EX
LB
GB
IW
PS
SB
SJ
UB
Max
Avg


In [20]:
 df_odds

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,SB_prob_w,SB_prob_l,SJ_prob_w,SJ_prob_l,UB_prob_w,UB_prob_l,Max_prob_w,Max_prob_l,Avg_prob_w,Avg_prob_l
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.797023,0.202977,0.658703,0.341297
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.797023,0.202977,0.658703,0.341297
2,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.797023,0.202977,0.658703,0.341297
3,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.797023,0.202977,0.658703,0.341297
4,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.797023,0.202977,0.658703,0.341297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54903,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Nadal R.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.690377,0.309623,0.675115,0.324885
54904,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.479070,0.520930,0.497382,0.502618
54905,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Tsitsipas S.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.271845,0.728155,0.281780,0.718220
54906,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Thiem D.,...,0.598433,0.401567,0.66448,0.33552,0.661114,0.338886,0.540541,0.459459,0.536458,0.463542


In [21]:
def logit(p):
    if p == 0 or p == 1:# or pd.isna(p):
        return np.nan
    return np.log(p / (1 - p))

def inverse_logit(y):
    if pd.isna(y):
        return np.nan
    return np.round(np.exp(y) / (1 + np.exp(y)),2)

In [22]:
bookmakers = ['B365', 'B&W', 'CB', 'EX', 'LB', 'GB', 'IW', 'PS', 'SB', 'SJ', 'UB', 'Max', 'Avg']

In [23]:
for bookmaker in bookmakers :
    df_odds[f'{bookmaker}_logit_prob_w'] = df_odds[f'{bookmaker}_prob_w'].apply(logit)

df_odds['consensus_logit_prob_w'] = df_odds[[f'{bookmaker}_logit_prob_w' for bookmaker in ['B365', 'B&W', 'CB', 'EX', 'LB', 'GB', 'IW', 'PS', 'SB', 'SJ', 'UB', 'Max', 'Avg']]].mean(axis=1)

df_odds['consensus_prob_w'] = df_odds['consensus_logit_prob_w'].apply(inverse_logit)

In [24]:
df_odds

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,GB_logit_prob_w,IW_logit_prob_w,PS_logit_prob_w,SB_logit_prob_w,SJ_logit_prob_w,UB_logit_prob_w,Max_logit_prob_w,Avg_logit_prob_w,consensus_logit_prob_w,consensus_prob_w
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,0.322875,0.35632,0.764754,0.398941,0.683321,0.668264,1.367790,0.657518,0.616162,0.65
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,0.322875,0.35632,0.764754,0.398941,0.683321,0.668264,1.367790,0.657518,0.616162,0.65
2,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,...,0.322875,0.35632,0.764754,0.398941,0.683321,0.668264,1.367790,0.657518,0.616162,0.65
3,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,...,0.322875,0.35632,0.764754,0.398941,0.683321,0.668264,1.367790,0.657518,0.616162,0.65
4,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,...,0.322875,0.35632,0.764754,0.398941,0.683321,0.668264,1.367790,0.657518,0.616162,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54903,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Nadal R.,...,0.322875,0.35632,0.852423,0.398941,0.683321,0.668264,0.801880,0.731413,0.584080,0.64
54904,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,...,0.322875,0.35632,-0.178590,0.398941,0.683321,0.668264,-0.083770,-0.010471,0.329810,0.58
54905,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Tsitsipas S.,...,0.322875,0.35632,-1.036577,0.398941,0.683321,0.668264,-0.985284,-0.935651,0.047112,0.51
54906,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Thiem D.,...,0.322875,0.35632,0.132172,0.398941,0.683321,0.668264,0.162519,0.146093,0.392808,0.60


In [25]:
df_odds['consensus_logit_prob_w'][(df_odds['consensus_logit_prob_w'] > 0.45) & (df_odds['consensus_logit_prob_w'] < 0.5)].shape

(2778,)

In [26]:
# # Store cumulative returns
# cumulative_returns = {}

# strategies = {
#     'Smallest Odd': smallest_odd_strategy,
#     'Best ATP Ranking': best_ranking_strategy,
#     'Random': random_strategy
# }

# for bookmaker in bookmakers:
#     for strategy_name, strategy_func in strategies.items():
#         key = f'{strategy_name} ({bookmaker})'
#         if strategy_name == 'Random':
#             # Average over multiple random runs for stability
#             random_returns = [evaluate_strategy(strategy_func) for _ in range(100)]
#             returns = np.mean(random_returns, axis=0)
#         else:
#             returns = evaluate_strategy(strategy_func, bookmaker)
#         cumulative_returns[key] = returns

# # Convert to DataFrame for plotting
# returns_df = pd.DataFrame(cumulative_returns)

# # Plot cumulative returns
# plt.figure(figsize=(14, 8))
# for col in returns_df.columns:
#     plt.plot(returns_df[col], label=col)
# plt.xlabel('Match Number')
# plt.ylabel('Cumulative Return')
# plt.title('Comparison of Betting Strategies for Two Bookmakers')
# plt.legend()
# plt.show()

In [27]:
all_years_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
       'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
       'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL'],
      dtype='object')

In [28]:
all_years_df.B365W.unique()

array([   nan,  1.1  ,  1.444,  2.625,  1.533,  1.143,  2.2  ,  1.8  ,
        1.571,  2.   ,  8.   ,  1.125,  2.1  ,  1.5  ,  1.364,  1.03 ,
        1.222,  1.167,  2.75 ,  1.667,  1.615,  1.286,  1.25 ,  1.071,
        2.375,  1.333,  1.909,  3.   ,  2.5  ,  1.4  ,  1.05 ,  1.04 ,
        7.5  ,  1.727,  7.   ,  2.25 ,  9.   ,  3.75 ,  1.111,  3.5  ,
        1.2  ,  4.333,  4.   ,  1.833,  4.5  ,  3.25 ,  1.083,  6.5  ,
        5.   ,  1.3  ,  1.063,  5.5  ,  2.875,  6.   ,  1.02 ,  1.118,
        1.36 ,  2.62 ,  1.28 ,  1.57 ,  1.44 ,  1.22 ,  1.61 ,  1.72 ,
        1.07 ,  1.06 ,  1.83 ,  1.66 ,  1.08 ,  1.33 ,  2.37 ,  1.53 ,
        1.14 ,  4.33 ,  1.16 ,  1.12 ,  2.87 ,  1.182,  3.4  ,  1.056,
        1.9  ,  1.11 ,  1.91 ,  1.67 ,  1.73 ,  2.63 ,  1.29 ,  1.62 ,
        2.38 ,  1.015,  1.01 ,  1.55 ,  1.19 ,  3.2  ,  3.39 ,  1.39 ,
        1.   ,  8.5  ,  1.38 ,  1.68 ,  2.79 ,  1.18 , 15.   , 12.   ,
       10.   ,  1.7  ,  2.4  , 13.   , 11.   ,  1.005,  2.15 ,  1.77 ,
      

In [29]:
all_years_df.B365L.unique()

array([    nan,   6.5  ,   2.625,   1.444,   2.375,   5.   ,   1.615,
         1.909,   2.25 ,   1.727,   1.063,   5.5  ,   1.667,   2.5  ,
         2.875,  10.   ,   4.   ,   4.5  ,   1.4  ,   2.1  ,   2.2  ,
         3.5  ,   3.75 ,   7.5  ,   1.533,   3.25 ,   1.8  ,   1.364,
         1.5  ,   2.75 ,   8.5  ,   9.   ,   1.071,   3.   ,   2.   ,
         1.083,   1.571,   1.04 ,   1.25 ,   6.   ,   1.286,   1.222,
         1.2  ,   1.833,   4.333,   1.167,   1.333,   7.   ,   1.1  ,
         1.143,   3.4  ,   8.   ,   1.125,   3.6  ,   1.111,  11.   ,
         6.25 ,   1.33 ,   1.44 ,   1.72 ,   2.62 ,   1.16 ,   1.83 ,
         1.53 ,   1.28 ,   1.61 ,   1.36 ,   1.57 ,   2.37 ,   1.73 ,
         4.33 ,   2.87 ,   1.14 ,   1.66 ,   1.3  ,   1.9  ,   1.22 ,
         3.2  ,   3.8  ,   2.38 ,   1.67 ,   2.88 ,   2.63 ,   1.62 ,
        12.   ,  13.   ,   2.3  ,   1.05 ,  15.   ,   1.08 ,   3.39 ,
         1.39 ,   1.12 ,  21.   ,   1.06 ,   1.19 ,   1.07 ,   1.11 ,
        17.   ,   1.

In [30]:
# # Function to calculate ROI
# def calculate_roi(data, win_col, lose_col):
#     smallest_odd = 100 * (data[win_col][data[win_col] < data[lose_col]].sum() - len(data)) / len(data)
#     best_ranking = 100 * (data[win_col][data.WRank < data.LRank].sum() - len(data)) / len(data)
#     random_betting = 100 * (data.sample(int(len(data) / 2))[win_col].sum() - len(data)) / len(data)
#     return smallest_odd, best_ranking, random_betting

# # Bookmaker columns
# bookmakers = ['B365W', 'B&WW', 'CBW', 'EXW', 'LBW', 'GBW', 'IWW', 'PSW', 'SBW', 'SJW', 'UBW', 'MaxW', 'AvgW']

# # Calculating ROI for all bookmakers
# values = []
# labels = []
# for bookmaker in bookmakers:
#     win_col = f'{bookmaker}'
#     lose_col = f'{bookmaker[:-1]}L'
#     smallest_odd, best_ranking, random_betting = calculate_roi(all_years_df, win_col, lose_col)
#     values.extend([smallest_odd, best_ranking, random_betting])
#     labels.extend([
#         f"{bookmaker[:-1]}\nsmallest odds strategy",
#         f"{bookmaker[:-1]}\nbest ranking strategy",
#         f"{bookmaker[:-1]}\nhead or tail betting"
#     ])

# # Plot configuration
# xaxis_label = "Return on investment (ROI) in %"
# title = "Betting on all ATP matches since 2011"

# # Assuming `basic_horizontal_barplot` is defined elsewhere
# basic_horizontal_barplot(values, labels, xaxis_label, title, [0, -8], (15, 10))

In [31]:
# # Plot cumulative returns
# plt.figure(figsize=(14, 8))
# for col in returns_df.columns:
#     plt.plot(returns_df[col], label=col)
# plt.xlabel('Match Number')
# plt.ylabel('Cumulative Return')
# plt.title('Comparison of Betting Strategies for Two Bookmakers')
# plt.legend()
# plt.show()

In [32]:
all_years_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
       'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
       'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL'],
      dtype='object')

In [33]:
df_odds['LRank'].unique()

array([7.700e+01, 5.600e+01, 6.550e+02, 8.700e+01, 1.980e+02, 1.050e+02,
       5.100e+01, 2.180e+02, 1.290e+02, 3.510e+02, 2.100e+01, 1.550e+02,
       1.220e+02, 7.900e+01, 2.000e+01, 8.800e+01, 6.500e+01, 6.300e+01,
       5.800e+01, 1.340e+02, 2.080e+02, 1.420e+02, 8.100e+01, 2.270e+02,
       2.600e+01, 1.100e+01, 8.400e+01, 5.500e+01, 1.500e+01, 4.000e+01,
       5.000e+00, 1.198e+03, 1.240e+02, 1.200e+02, 6.700e+01, 2.000e+02,
       8.900e+01, 1.030e+02, 1.350e+02, 1.150e+02, 1.370e+02, 4.140e+02,
       1.000e+02, 3.420e+02, 2.000e+00, 2.380e+02, 1.130e+02, 9.400e+01,
       1.110e+02, 1.270e+02, 1.160e+02, 1.600e+02, 1.250e+02, 1.450e+02,
       9.900e+01, 1.170e+02, 1.620e+02, 1.630e+02, 1.210e+02, 1.040e+02,
       1.400e+01, 1.280e+02, 3.600e+01, 1.000e+05, 2.190e+02, 3.500e+01,
       7.800e+01, 1.740e+02, 5.900e+01, 2.500e+01, 1.010e+02, 1.380e+02,
       8.600e+01, 2.850e+02, 9.300e+01, 6.200e+01, 5.000e+01, 7.600e+01,
       6.000e+01, 7.300e+01, 2.110e+02, 7.100e+01, 

In [34]:
df_odds['actual_winner'] = (df_odds['WRank'] < df_odds['LRank'])



predictions = (df_odds['consensus_prob_w'] > 0.5)

#valid_indices = df_odds['consensus_prob_w'].notna()

accuracy_score = accuracy(df_odds['actual_winner'], predictions)
accuracy_score



0.723537553726233

In [35]:
df_odds['actual_winner'].value_counts()

True     35878
False    19030
Name: actual_winner, dtype: int64

In [36]:
calibration(df_odds['actual_winner'],df_odds['consensus_prob_w'] )

0.9616263448352752

In [37]:
logloss(df_odds['actual_winner'],df_odds['consensus_prob_w'])

0.558100233713572